**Naive Bayes - IMDB movie reviews - Keras ad Sklearn**

Trong bài này chúng ta sẽ dùng thuật toán phân loại Naive Bayes cho bộ dữ liệu IMDB reviews sử dụng thư viện Keras và Sklearn.





**1. Chuẩn bị dữ liệu**

In [1]:
from keras.datasets import imdb
from keras.datasets.imdb  import get_word_index
from keras.preprocessing import sequence
import numpy as np
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn import metrics

**1.1. Tải dữ liệu IMDB movie reviews**

IMDB movie reviews là bộ dữ liệu chứa các review cho phim được phân làm 2 lớp: tiêu cực và tích cực. Dữ liệu được chia thành 2 tập: 25000 reviews cho việc huấn luyện và 25000 reviews cho việc kiểm tra. Mỗi tập sẽ chứa số reviews tiêu cực và tích cực bằng nhau.

X_train bên dưới sẽ là một ma trận, trong đó mỗi dòng của ma trận là một chuỗi tương ứng với 1 review phim. Ở đây mỗi reivew đã được tiền xử lý thành một chuỗi số tự nhiên, trong đó mỗi số tự nhiên đại diện cho một từ trong từ điển. 

y_train là một véc tơ nhãn gồm các con số 0 (tiêu cực) và 1 (tích cực). 

In [2]:
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

Ở đây biến top_words hạn chế các từ của review chứa trong 10000 từ ngữ thông dụng. Bộ dữ liệu đầy đủ chứa review bao gồm các từ trong khoảng 90000 từ ngữ thông dụng. 

**1.2. Xem xét dữ liệu**

In [3]:
print('X_train_shape:', X_train.shape)
print('y_train_shape:', y_train.shape)
print('X_test_shape:', X_test.shape)
print('y_test_shape:', y_test.shape)

X_train_shape: (25000,)
y_train_shape: (25000,)
X_test_shape: (25000,)
y_test_shape: (25000,)


Chúng ta sẽ download bộ từ điển từ ngữ thông dụng và chứa trong biến word_indices. 

In [4]:
word_indices = get_word_index(path="imdb_word_index.json")
len(word_indices)

88584

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_indices.items()])

Một review phim sẽ có dạng như sau:

In [6]:
print(X_train[0])
print(len(X_train[0]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
218


Chúng ta sẽ giải mã review phim vừa rồi để xem xét các từ ngữ trong review phim đó.

In [7]:
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in X_train[0]])

In [8]:
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

Dấu hỏi (?) ở đoạn review trên tương ứng với một từ không nằm trong 10000 từ thông dụng.

**Chuẩn bị dữ liệu**

Chúng ta không thể dụng một chuỗi số tự nhiên trong thuật toán phân lớp Naive Bayes, chính vì thế chúng ta cần biến đổi dữ liệu về định dạng cần thiết.

Chúng ta sẽ One-hot-encode chuỗi số tự nhiên và biến những chuỗi này thành những vec tơ những con số 0 và 1. Việc này tương ứng với việc biến một chuỗi review thành một véc tơ với độ dài bằng độ dài của top_words, là 10000. Sô 1 ở vị trí i tương ứng với từ i xuất hiện trong review, số 0 tương ứng với từ i không xuất hiện trong review. 

In [9]:
def vectorize_sequences(sequences, dimension= top_words):
    results = np.zeros((len(sequences), dimension))    # Creates an all zero matrix of shape (len(sequences),10000)
    for i,sequence in enumerate(sequences):
        results[i,sequence] = 1                        # Sets specific indices of results[i] to 1s
    return results

In [10]:
# Vectorize training Data
X_train = vectorize_sequences(X_train)

# Vectorize testing Data
X_test = vectorize_sequences(X_test)

Ta cũng biến đổi tập nhãn về định dạng số thực.

In [11]:
y_train = np.asarray(y_train).astype('float32')
y_test  = np.asarray(y_test).astype('float32')

Hãy xem một dữ liệu trong tập huấn luyện như thế nào.

In [12]:
X_train[0,0:150]

array([0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.])

Chúng ta hãy coi trong chuỗi X_train[0] có bao nhiều từ xuất hiện. 

In [13]:
sum(X_train[0,0:10000])

120.0

Các bạn có thể giải thích tại sao chiều dài của X_train[0] ban đầu là 285 mà sau khi xử lý thì chỉ còn 120 từ không?

**2. Xây dựng mô hình huấn luyện Naive Bayes**

Chúng ta sẽ xem xét thuật toán phân lớp Naive Bayes với ba loại phân phối khác nhau: Gaussian Naive Bayes, Multinomial Naive Bayes và Bernoulli Naive Bayes. Chi tiết về 3 loại phân phối này có thể xem [ở đây](https://machinelearningcoban.com/2017/08/08/nbc/).

**2.1 Phân phối Gaussian Naive Bayes**

In [14]:
model = GaussianNB()

In [15]:
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

Chúng ta đã sẵng sàng để dự doán trên tập X_test và in ra độ chính xác.

In [16]:
y_pred = model.predict(X_test)


In [17]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.69644


**2.2. Phân phối Multinomial Naive Bayes**

In [18]:
model = MultinomialNB()

In [19]:
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
y_pred = model.predict(X_test)

In [21]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.83936


**2.3. Phân phối Bernoulli Naive Bayes**

In [22]:
model = BernoulliNB()

In [23]:
model.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [24]:
y_pred = model.predict(X_test)

In [25]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.84044


Bạn có nhận xét gì về 3 độ chính xác trên? Các bạn có thể thử huấn luyện bộ dữ liệu trên dùng thuật toán Logistic regression và xem xét kết quá.